# From probability to the bidding price

Since the goal is to "optimize the cost per customer while having 4% customer rate over all ads shown". The simpliest intuition is to bid more on valuable customers. If we forget about the $4%$ constraint for a second, to decrease the cost per sold, we only need to consider the probability `P(sold|click)` for a customer as the company only need to pay when clicks happen
$$P(sold|click)=\frac{P(\text{sold and click})}{P(click)}=\frac{P(sold)}{P(click)}$$

Since current cost per customer is around 24.0 dollars per customer and the sold rate (sold/shown) is 7.83\% and average P(sold|click)=41.69\%, if we set the average as the baseline for 10 dollars and assume we invest linearly with the probability `P(sold|click)`, we'll have cost per customer even higher 24.19 dollars.

In [2]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# read the data as dataframe
df = pd.read_csv("Root_insurance_data.csv")

cols = df.columns
df_unique = df[cols[:4]].drop_duplicates()
print(len(df_unique))
df_unique.head()

35


,Currently Insured,Number of Vehicles,Number of Drivers,Marital Status
0,unknown,2,1,M
1,Y,3,1,M
2,Y,3,2,M
3,N,3,2,S
4,unknown,2,2,S


In [4]:
from sklearn.preprocessing import FunctionTransformer

def one_hot_encode(df):
    df_copy = df.copy()
    
    hot_encoding = pd.get_dummies(df_copy['Currently Insured'])
    df_copy[hot_encoding.columns] = hot_encoding[hot_encoding.columns]
    
    return df_copy

one_hot = FunctionTransformer(one_hot_encode)

In [5]:
df = one_hot.transform(df)
df.loc[df.click == True, "Click"] = int(1)
df.loc[df.click == False, "Click"] = int(0)
df.loc[df['Marital Status'] == "M", "Marital"] = int(1)
df.loc[df['Marital Status'] == "S", "Marital"] = int(0)
# convert to integers
df = df.astype({"Click": int}) 
#df.loc[df.click == True][:5]
df.describe()

,Number of Vehicles,Number of Drivers,bid,rank,policies_sold,N,Y,unknown,Click,Marital
count,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.00000
mean,1.965400,1.499900,10.0,3.184100,0.078300,0.344400,0.34190,0.313700,0.187800,0.51910
std,0.807755,0.500025,0.0,1.377242,0.268657,0.475196,0.47437,0.464019,0.390572,0.49966
min,1.000000,1.000000,10.0,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
25%,1.000000,1.000000,10.0,2.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
50%,2.000000,1.000000,10.0,3.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.00000
75%,3.000000,2.000000,10.0,4.000000,0.000000,1.000000,1.00000,1.000000,0.000000,1.00000
max,3.000000,2.000000,10.0,5.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000


In [6]:
df_unique = one_hot.transform(df_unique)
df_unique.loc[df['Marital Status'] == "M", "Marital"] = int(1)
df_unique.loc[df['Marital Status'] == "S", "Marital"] = int(0)
#df.loc[df.click == True][:5]
df_unique.describe()

,Number of Vehicles,Number of Drivers,N,Y,unknown,Marital
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,1.971429,1.485714,0.342857,0.342857,0.314286,0.514286
std,0.821967,0.507093,0.481594,0.481594,0.471008,0.507093
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000
max,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
X = df[["Number of Vehicles", "Number of Drivers", "N","Y","unknown","Marital"]].copy()
X_unique = df_unique[["Number of Vehicles", "Number of Drivers", "N","Y","unknown","Marital"]].copy()
y_rank = df["rank"].copy()
y_click = df["Click"].copy()
y_sold = df["policies_sold"].copy()

In [17]:
# frequency count for each type of customer
ClickCount = [0] * len(X_unique)
SoldCount = [0] * len(X_unique)
FreqCount = [0] * len(X_unique)

features = list(X_unique.columns)
featuredict = {}
for i in range(len(X_unique)):
    temp = list(X_unique.iloc[i])
    string = ''
    for num in temp:
        string += str(num)
    if string not in featuredict:
        featuredict[string] = i

for i in range(len(X)):
    temp = list(X.iloc[i])
    string = ''
    for num in temp:
        string += str(num)
    FreqCount[featuredict[string]] += 1
    if y_click[i]:
        ClickCount[featuredict[string]] += 1
        if y_sold[i]:
            SoldCount[featuredict[string]] += 1

ClickCount = np.array(ClickCount)
SoldCount = np.array(SoldCount)
FreqCount = np.array(FreqCount)       
click_prob = ClickCount/FreqCount # probability from requency count
sold_prob = SoldCount/FreqCount

In [21]:
#print(click_prob)
#print(sold_prob)
PSC = sold_prob/click_prob
PSC

array([0.48809524, 0.19047619, 0.11111111, 0.23076923, 0.38181818,
       0.4       , 0.38095238, 0.3125    , 0.43835616, 0.54285714,
       0.1875    , 0.26666667, 0.40506329, 0.31578947, 0.31578947,
       0.6       , 0.56      , 0.31168831, 0.36363636, 0.27659574,
       0.52380952, 0.4296875 , 0.48717949, 0.40740741, 0.58139535,
       0.45454545, 0.36231884, 0.46875   , 0.2972973 , 0.20833333,
       0.36363636, 0.52678571, 0.54330709, 0.41891892, 0.3880597 ])

### What if we take extreme cases? 

In the limit of infinite budget and customer samples, we should invest all the budget to the most valuable customer so as to obtain the best cost per customer. However, the limited budget and customer samples requires us invest on more customer with lower bound given by the 4% customer rate. Compared to previous strategy, the linear relation with the average `P(sold|click)` rate might be two slow. Thus, here we try a exponential function function 
$$ B= 1+e^{C (P-\bar{P})}$$
where the bidding price has minimum 1 dollar. The coefficient in the exponent $C\equiv 20$. $\bar{P}$ is the average of $P(S|C)$.

In [43]:
PSC = sold_prob/click_prob
PSCmean, C = 0.2, 7.4 #np.mean(PSC) # 0.5, 29;
biddings = 1 + np.round(np.exp(C*(PSC-PSCmean))) #np.round(1+Cmax/(1+np.exp(-C*(PSC-PSCmean))))
biddings

array([ 9.,  2.,  2.,  2.,  5.,  5.,  5.,  3.,  7., 14.,  2.,  3.,  6.,
        3.,  3., 20., 15.,  3.,  4.,  3., 12.,  6.,  9.,  6., 18.,  8.,
        4.,  8.,  3.,  2.,  4., 12., 14.,  6.,  5.])

In [44]:
# Add new bidding to dataframe
df_unique['newbid'] = biddings
newcols = list(df_unique.columns)
newcols = newcols[1:3] + newcols[4:8]
featuredict = {}
for i in range(len(df_unique)):
    temp = list(df_unique.iloc[i][newcols])
    string = ''
    for num in temp:
        string += str(num)
    if string not in featuredict:
        featuredict[string] = df_unique.iloc[i]['newbid']

df_copy = df.copy()
newbid = []
for i in range(len(df_copy)):
    temp = list(df_copy.iloc[i][newcols])
    string = ''
    for num in temp:
        string += str(num)
    newbid.append(featuredict[string])
    
df_copy['newbid'] = newbid

In [45]:
import random

budget = 10*np.sum(df["Click"])
budget

18780

In [49]:
trials = 10
showList = [0] * trials
costList = [0] * trials
soldList = [0] * trials
soldRateList = [0] * trials
CPCList = [0] * trials
OldCPCList = [0] * trials

for trial in range(trials):
    costs = 0
    sold = 0
    clicks = 0
    shown = 0
    random.seed(trial)
    while costs < budget:
        shown += 1
        ind = random.randint(0, len(df_copy)-1)
        if df_copy.iloc[ind]['click'] and df_copy.iloc[ind]['newbid'] > 0:
            costs += df_copy.iloc[ind]['newbid']
            clicks += 1
            if df_copy.iloc[ind]['policies_sold']:
                sold += 1
    showList[trial] = shown
    costList[trial] = costs
    soldList[trial] = sold
    soldRateList[trial] = sold/shown
    CPCList[trial] = costs/sold
    OldCPCList[trial] = 10*clicks/sold
    print(trial)
    print("showned times", shown, "total", costs, "sold",sold, "sold rate", sold/shown)
    print("cost per customer", costs/sold, "Old", 10*clicks/sold,"\n")

0
showned times 14631 total 18783.0 sold 1060 sold rate 0.07244890984895086
cost per customer 17.719811320754715 Old 24.547169811320753 

1
showned times 13336 total 18791.0 sold 1066 sold rate 0.07993401319736053
cost per customer 17.627579737335836 Old 23.583489681050658 

2
showned times 13741 total 18780.0 sold 1039 sold rate 0.07561312859326104
cost per customer 18.075072184793072 Old 24.619826756496632 

3
showned times 13971 total 18786.0 sold 1056 sold rate 0.07558514064848615
cost per customer 17.789772727272727 Old 24.6875 

4
showned times 14310 total 18796.0 sold 1092 sold rate 0.07631027253668764
cost per customer 17.21245421245421 Old 23.498168498168496 

5
showned times 13774 total 18781.0 sold 1068 sold rate 0.07753738928415856
cost per customer 17.585205992509362 Old 24.204119850187265 

6
showned times 13839 total 18788.0 sold 1101 sold rate 0.0795577715152829
cost per customer 17.064486830154404 Old 23.115349682107176 

7
showned times 13452 total 18780.0 sold 1104 s

In [51]:
df_bidding = pd.DataFrame({"number of shown": showList,
                          "total cost": costList,
                         "number of sold": soldList,
                          "sold rate": soldRateList,
                          "cost per customer": CPCList,
                          "old CPC": OldCPCList})
df_bidding

,number of shown,total cost,number of sold,sold rate,cost per customer,old CPC
0,14631,18783.0,1060,0.072449,17.719811,24.547170
1,13336,18791.0,1066,0.079934,17.627580,23.583490
2,13741,18780.0,1039,0.075613,18.075072,24.619827
3,13971,18786.0,1056,0.075585,17.789773,24.687500
4,14310,18796.0,1092,0.076310,17.212454,23.498168
5,13774,18781.0,1068,0.077537,17.585206,24.204120
6,13839,18788.0,1101,0.079558,17.064487,23.115350
7,13452,18780.0,1104,0.082070,17.010870,22.934783
8,13593,18781.0,1100,0.080924,17.073636,23.345455
9,13481,18782.0,1041,0.077220,18.042267,24.409222


In [53]:
df_bidding["cost per customer"].describe()

count    10.000000
mean     17.520116
std       0.404135
min      17.010870
25%      17.108341
50%      17.606393
75%      17.772282
max      18.075072
Name: cost per customer, dtype: float64